In [ ]:
!pip install pandas==1.4.4 

In [ ]:
!pip install catboost

In [ ]:
!pip install openai

In [ ]:
!pip install levenshtein

In [ ]:
!pip install datasketch

In [ ]:
!pip install SetSimilaritySearch

Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import json

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier, Pool
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score 

from sklearn.metrics.pairwise import cosine_similarity
from SetSimilaritySearch import all_pairs
from Levenshtein import ratio
from numpy.linalg import norm

import openai

from collections import Counter

Подготовка данных с помощью следующих функций:

Функция блокировки входных данных

In [ ]:
def blocking_func() -> pd.DataFrame:
    """
    Данная функция не участвует в действующем алгоритме, но является незаменимой 
    частью данного процесса - получение данных и сокращения их количетсва.
    
    Данная функция реализует следующий функцтонал:
    1) Принимает данные из источника 
    2) Обрабатывает данные (приведение типов итд)
    3) Отбирает актуальыные данные (current_date()-1)
    4) Разделяет по количеству продукции (кол-во продукции > порогового значения)
    5) Выбирает товар в наличие/нет (True/False) 
    6) Разделяет по категориям товара + подкатеории товара 

    Parameters
    ----------
    Данные из источника

    Returns
    -------
    pd.Dataframe() 
    (с данными по товарам определенного количества, категории и 
    актуальности)

    """
    pass

Функция разметки разных тайтлов с помощью трансформера

In [ ]:
def gpt_label(
    df:pd.DataFrame
    ) -> pd.DataFrame:
    """
        Данная функция не участвует в действующем алгоритме, но является дополнительным инструментом,
        который способен различать тайтлы товаров. На основе данной функции можно реализовать свой трансформер,
        следуя архитектуре, например, chatgpt, который в дальнейшем спообен внести дополнительную фичу различия товаров по их тайтлам
        Parameters
        ----------
        df : {pandas.DataFrame}

        Returns
        -------
        df : pd.Dataframe() - датафрейм с дополнительной фичей в качестве разметки с помощью трансформера (например, chatgpt)
    """
    openai.api_key = "sk-???"
    
    title_1 = "Колодка TDM Electric четырехместная без заземл"
    title_2 = "Клавиатура черная с черной рамкой для 25-011879"

    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
              {"role": "system", "content": "You are a chatbot"},
              {"role": "user", "content": f"{title_1} and {title_2} are similar? Yes or No?"},
          ]
    )

    result = ''
    for choice in response.choices:
        result += choice.message.content

    df["gpt_label"] = result

    return df

Блок функции-извлекателей 

In [ ]:
def extract_category_3lvl(
    df:pd.DataFrame
) -> pd.DataFrame:
    """
        Данная функция является извлекателем категории на уровне равном 3 из исходного датафрейма
        
        Parameters
        ----------
        df : {pandas.DataFrame}
        
        Returns
        ----------
        df : {pandas.DataFrame} датафрейм с дополнительной фичой - category_3lvl
    """

    categories_arr = []
    for i, _ in enumerate(df["categories"]):
        try:
            categories_arr.append(json.loads(df['categories'][i])['3']) 
        except:
            categories_arr.append("")
      
    df["category_3lvl"] = categories_arr

    return df 

In [ ]:
def extract_brand(
    df:pd.DataFrame
) -> pd.DataFrame:
    """
        Данная функция является извлекателем бренда из исходного датафрейма
        
        Parameters
        ----------
        df : {pandas.DataFrame}
        
        Returns
        ----------
        df : {pandas.DataFrame} датафрейм с дополнительной фичой - brand
    """

    brand_arr = []
    for i, _ in enumerate(df["characteristic_attributes_mapping"]):
        try:
            brand_arr.append(json.loads(df['characteristic_attributes_mapping'][i])['Бренд'][0]) 
        except:
            brand_arr.append("")

    df["brand"] = brand_arr

    return df 

Функция сопоставления id и их данных попарно 

In [ ]:
def match_products(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
       Данная функция сопоставляет id продуктов их дополнительные атрибуты согласно 
       данным из файла с парами
       
       Parameters
       ----------
       df : {pandas.DataFrame}
        
       Returns
       ----------
       df : {pandas.DataFrame} датафрейм попарно соединенный согласно файлу с парами
       
    """

    pairs_df = pd.read_parquet("train_pairs.parquet") # train_pairs
    pairs_df = pairs_df[["variantid1", "variantid2", "target"]] # , "target"

    df_ready = pd.DataFrame()
    for i in range(len(pairs_df)): # DELETE .iloc[:20]
        # print("i -", i)
        prod_ind_arr = pairs_df.iloc[i, 0:2].to_list() # [52076340,290590137]      # non actual/change columns (.iloc[i, 1:] - for train / .iloc[i, 0:] - for test)
        # print(prod_ind_arr)

        if df[df["variantid"] == int(prod_ind_arr[0])].empty == False: # df[df["variantid"] == 52076340] 
            df_id1 = df[df["variantid"] == prod_ind_arr[0]] \
                    .reset_index() \
                    .rename(columns={
                        "index": "index1",
                        "variantid": "variantid1", 
                        "name": "name1", 
                        "categories": "categories1",
                        "color_parsed":"color_parsed1",
                        "pic_embeddings_resnet_v1": "pic_embeddings_resnet_v1_1", 
                        "main_pic_embeddings_resnet_v1": "main_pic_embeddings_resnet_v1_1",
                        "name_bert_64": "name_bert_64_1",
                        "characteristic_attributes_mapping": "characteristic_attributes_mapping1",
                        "brand": "brand1",
                        "category_3lvl": "category_3lvl_1"
                        }) 
        else:
            df_id1 = pd.DataFrame()

        if df[df["variantid"] == int(prod_ind_arr[1])].empty == False: # df[df["variantid"] == 290590137] 
            df_id2 = df[df["variantid"] == prod_ind_arr[1]] \
                    .reset_index() \
                    .rename(columns={
                        "index": "index2",
                        "variantid": "variantid2", 
                        "name": "name2", 
                        "categories": "categories2",
                        "color_parsed":"color_parsed2",
                        "pic_embeddings_resnet_v1": "pic_embeddings_resnet_v1_2", 
                        "main_pic_embeddings_resnet_v1": "main_pic_embeddings_resnet_v1_2",
                        "name_bert_64": "name_bert_64_2",
                        "characteristic_attributes_mapping": "characteristic_attributes_mapping2",
                        "brand": "brand2",
                        "category_3lvl": "category_3lvl_2"
                        }) 
        else:
            df_id2 = pd.DataFrame()

        df_id1_id2 = pd.concat([df_id1, df_id2], axis=1)

        df_ready = pd.concat([df_ready, df_id1_id2], axis=0)

    return df_ready.reset_index(drop=True)

Функции сходства данных

In [ ]:
def cos_similarity_num(
    df:pd.DataFrame
) -> pd.DataFrame:
    # Vectors input
    """
        Данная функция предназначена для расчета косинусной схожести объектов 
        для числовых векторов
        
        Parameters
        ----------
        df : {pandas.DataFrame}
        
        Returns
        ----------
        df : {pandas.DataFrame} датафрейм с дополнительными фичами схожести 
        числовых векторов - ["cos_sim_pic_emb", "cos_sim_main_pic_emb", "cos_sim_name_bert"]
        
    """

    columns = [("pic_embeddings_resnet_v1_1", "pic_embeddings_resnet_v1_2"),
             ("main_pic_embeddings_resnet_v1_1", "main_pic_embeddings_resnet_v1_2"),
             ("name_bert_64_1", "name_bert_64_2") 
             ]
    new_col = ["cos_sim_pic_emb", "cos_sim_main_pic_emb", "cos_sim_name_bert"]
  
    for i, k in enumerate(columns):
        cos_sim_arr = []
        for j in range(len(df)):
            try:
                col_1 = k[0]
                col_2 = k[1]
                cos_sim_arr.append(
                    np.dot(tst_df[col_1][j][0], tst_df[col_2][j][0]) / (norm(tst_df[col_1][j][0]) * norm(tst_df[col_2][j][0]))
                )
            except:
                cos_sim_arr.append(0)

        df[new_col[i]] = cos_sim_arr
    
    return df

In [ ]:
def levenshtein_similarity(
    df:pd.DataFrame
) -> pd.DataFrame:
    # Words input
    """
        Данная функция предназначена для расчета расстояния Левинштейна схожести объектов 
        для текстовых атрибутов
        
        Parameters
        ----------
        df : {pandas.DataFrame}
        
        Returns
        ----------
        df : {pandas.DataFrame} датафрейм с дополнительными фичами схожести 
        текстовых атрибутов - ["levenshtein_sim_brand", "levenshtein_sim_category_3lvl", "levenshtein_sim_name"]
    
    """

    columns = [
      ("brand1", "brand2"),
      ("category_3lvl_1", "category_3lvl_2"),
      ("name1", "name2")
    ]
    new_col = ["levenshtein_sim_brand", "levenshtein_sim_category_3lvl", "levenshtein_sim_name"]

    for i, k in enumerate(columns):
        levenshtein_sim_arr = []
        col_1 = k[0]
        col_2 = k[1]
        for ind, val in zip(df[col_1].to_list(), df[col_2].to_list()):
            try:
                levenshtein_sim_arr.append(
                    ratio(ind, val)
                )
            except:
                levenshtein_sim_arr.append(0)

        df[new_col[i]] = levenshtein_sim_arr

    return df

In [ ]:
def jaccard_similarity(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
        Данная функция предназначена для расчета расстояния Джаккарта схожести объектов 
        для текстовых атрибутов
        
        Parameters
        ----------
        df : {pandas.DataFrame}
        
        Returns
        ----------
        df : {pandas.DataFrame} датафрейм с дополнительными фичами схожести 
        текстовых атрибутов - ["jaccard_sim_brand", "jaccard_sim_category_3lvl", "jaccard_sim_name"]
    
    """

    columns = [
      ("brand1", "brand2"),
      ("category_3lvl_1", "category_3lvl_2"),
      ("name1", "name2")
  ]
    new_col = ["jaccard_sim_brand", "jaccard_sim_category_3lvl", "jaccard_sim_name"]
  
    for i, k in enumerate(columns):
        jacc_sim_arr = []
        col_1 = k[0]
        col_2 = k[1]
        for j in range(len(df)):
            try:
                doc1 = df[col_1][j]
                doc2 = df[col_2][j]

                words_doc1 = set(doc1.lower().split()) 
                words_doc2 = set(doc2.lower().split())

                intersection = words_doc1.intersection(words_doc2)
                union = words_doc1.union(words_doc2)

                jacc_sim_arr.append(float(len(intersection)) / len(union))
            except:
                jacc_sim_arr.append(0)

        df[new_col[i]] = jacc_sim_arr
    
    return df

In [ ]:
def containment_similarity(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
        Данная функция предназначена для расчета расстояния по содержанию схожести объектов 
        для текстовых атрибутов
        
        Parameters
        ----------
        df : {pandas.DataFrame}
        
        Returns
        ----------
        df : {pandas.DataFrame} датафрейм с дополнительными фичами схожести 
        текстовых атрибутов - ["containment_sim_brand", "containment_sim_category_3lvl", "containment_sim_name"]
    
    """

    columns = [
      ("brand1", "brand2"),
      ("category_3lvl_1", "category_3lvl_2"),
      ("name1", "name2")
      ]
    new_col = ["containment_sim_brand", "containment_sim_category_3lvl", "containment_sim_name"]
  
    for i, k in enumerate(columns):
        containment_sim_arr = []
        col_1 = k[0]
        col_2 = k[1]
        for j in range(len(df)):
            try:
                doc1 = df[col_1][j]
                doc2 = df[col_2][j]

                words_doc1 = set(doc1.lower().split()) 
                words_doc2 = set(doc2.lower().split())

                intersection = words_doc1.intersection(words_doc2)

                if len(words_doc1) > len(words_doc2):
                    first_set = len(words_doc1)
                else:
                    first_set = len(words_doc2)

                containment_sim_arr.append(float(len(intersection)) / first_set)
            except:
                containment_sim_arr.append(0)

        df[new_col[i]] = containment_sim_arr
  
    return df

In [ ]:
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    
def get_vectors(*strs):
    text = [t for t in strs]
    # print(text)
    vectorizer = CountVectorizer()
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

def tf_idf_cos_similarity(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
        Данная функция предназначена для расчета косинусной схожести объектов 
        для текстовых атрибутов на основе tf-idf
        
        Parameters
        ----------
        df : {pandas.DataFrame}
        
        Returns
        ----------
        df : {pandas.DataFrame} датафрейм с дополнительными фичами схожести 
        текстовых атрибутов - ["tf_idf_cos_sim_name", "tf_idf_cos_sim_category_3lvl"]
    
    """
  
    columns = [
      ("name1", "name2"),
      ("category_3lvl_1", "category_3lvl_2")
      ]
    new_col = ["tf_idf_cos_sim_name", "tf_idf_cos_sim_category_3lvl"]

    for i, k in enumerate(columns):
        tf_idf_cos_sim_arr = []
        col_1 = k[0]
        col_2 = k[1]
        for j in range(len(df)):
            try:
                tf_idf_cos_sim_arr.append(get_cosine_sim(df[col_1][j], df[col_2][j])[0][1])
            except:
                tf_idf_cos_sim_arr.append(0)
    
        df[new_col[i]] = tf_idf_cos_sim_arr
  
    return df

PipeLine Start

In [ ]:
train_df = pd.read_parquet("train_data.parquet") # test_data - так как данных тут меньше (для отладки функций)/ нужно train_data.parquet
train_df.head(3)

In [ ]:
pairs_df = pd.read_parquet("train_pairs.parquet")
pairs_df.head(5)

In [ ]:
tst_df = extract_category_3lvl(train_df)
tst_df = extract_brand(tst_df)
tst_df = match_products(tst_df)
tst_df = cos_similarity_num(tst_df)
tst_df = levenshtein_similarity(tst_df)
tst_df = jaccard_similarity(tst_df)
tst_df = containment_similarity(tst_df)
tst_df = tf_idf_cos_similarity(tst_df)

tst_df.head()

In [ ]:
tst_df["target"] = pairs_df["target"]
tst_df

In [ ]:
tst_df['color_parsed1'].fillna("['бесцветный']",inplace = True)
tst_df['color_parsed2'].fillna("['бесцветный']",inplace = True)

In [ ]:
# Выборка фичей 

columns_need = [
    'variantid1','variantid2', 
    'name1','name2', 
    'color_parsed1','color_parsed2',
#     'pic_embeddings_resnet_v1_1', 'pic_embeddings_resnet_v1_2',
    'main_pic_embeddings_resnet_v1_1', 'main_pic_embeddings_resnet_v1_2',
    'name_bert_64_1', 'name_bert_64_2',
    'category_3lvl_1','category_3lvl_2',
#     'brand1','brand2',
    'cos_sim_pic_emb', 'cos_sim_main_pic_emb', 'cos_sim_name_bert',
    'levenshtein_sim_brand', 'levenshtein_sim_category_3lvl',
    'levenshtein_sim_name', 'jaccard_sim_brand',
    'jaccard_sim_category_3lvl', 'jaccard_sim_name',
    'containment_sim_brand', 'containment_sim_category_3lvl',
    'containment_sim_name', 'tf_idf_cos_sim_name',
    'tf_idf_cos_sim_category_3lvl', 
    'target'
]

In [ ]:
mid_df = tst_df[columns_need]

In [ ]:
X = mid_df.iloc[:, 2: mid_df.shape[1]-1]
y = mid_df["target"]

In [ ]:
from pandas.api.types import is_numeric_dtype

def get_categorical_indicies(X):
    """
        Данная функция реализует поиск индексов, которые считаются типом "object"
        
        Parameters
        ----------
        X : датафрейм с фичами, на которых в дальнейшем будет обучаться модель
        
        Returns
        ----------
        cat_indicies : список индексов, которые считаются типом "object"
        
    """
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
        
    return cat_indicies

categorical_indicies = get_categorical_indicies(X)
categorical_indicies

In [ ]:
def convert_cats(X):
    """
        Данная функция реализует конвертацию признаков "object" -> "category"
        
        Parameters
        ----------
        X : датафрейм с фичами, на которых в дальнейшем будет обучаться модель
        
        Returns
        ----------
        -
    """
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        X[col] = X[col].astype('category')
        
convert_cats(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, 
                                                  y, 
                                                  test_size=0.2, 
                                                  random_state=0)

In [ ]:
# test_data_changed.csv - файл, который содержит пары товаров с их дополнительными
# атрибутами (реализация аналогична для train_data.parquet, только с даннами из test_data.parquet)
df_test = pd.read_csv("full_test_data_changed.csv")

In [ ]:
columns_need_test = [
    'variantid1','variantid2', 
    'name1','name2', 
    'color_parsed1','color_parsed2',
#     'pic_embeddings_resnet_v1_1', 'pic_embeddings_resnet_v1_2',
    'main_pic_embeddings_resnet_v1_1', 'main_pic_embeddings_resnet_v1_2',
    'name_bert_64_1', 'name_bert_64_2',
    'category_3lvl_1','category_3lvl_2',
#     'brand1','brand2',
    'cos_sim_pic_emb', 'cos_sim_main_pic_emb', 'cos_sim_name_bert',
    'levenshtein_sim_brand', 'levenshtein_sim_category_3lvl',
    'levenshtein_sim_name', 'jaccard_sim_brand',
    'jaccard_sim_category_3lvl', 'jaccard_sim_name',
    'containment_sim_brand', 'containment_sim_category_3lvl',
    'containment_sim_name', 'tf_idf_cos_sim_name',
    'tf_idf_cos_sim_category_3lvl', 
] 

X_test_start = df_test[columns_need_test]

In [ ]:
X_test_start['color_parsed1'].fillna("['бесцветный']",inplace = True)
X_test_start['color_parsed2'].fillna("['бесцветный']",inplace = True)

In [ ]:
X_test = X_test_start.iloc[:, 2:]

convert_cats(X_test)

ML модель 

Поиск оптимальных параметров

In [ ]:
pipe_catboost = Pipeline([
    ('scl', StandardScaler()),
    ('clf', CatBoostClassifier(random_state=1))
])

# model = CatBoostClassifier()
# param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

grid = {'clf__learning_rate': [ 0.03, 0.01, 0.1, 1.0],
        'clf__depth': [4, 6, 10],
        'clf__l2_leaf_reg': [2,  5, 7, 9]}

gs_catboost = GridSearchCV(estimator=pipe_catboost,
                           param_grid=grid,
                           cv=10, 
                           n_jobs=-1)

gs_catboost.fit(X_train.iloc[:5000], y_train.iloc[:5000])

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",gs_catboost.best_estimator_)
print("\n The best score across ALL searched params:\n",gs_catboost.best_score_)
print("\n The best parameters across ALL searched params:\n",gs_catboost.best_params_)

Реализация ML модели

In [ ]:
cls_catboost_v2 = CatBoostClassifier(random_state=1,
                                  loss_function='Logloss',
                                  # eval_metric='Accuracy',
                                  depth=7, 
                                  l2_leaf_reg=5, 
                                  learning_rate=0.1 # 0.05
                                 )
cls_catboost_v2.fit(X_train,y_train,
                 cat_features=categorical_indicies,
                 eval_set=(X_val, y_val)
                )

In [ ]:
predictions_v2 = cls_catboost_v2.predict_proba(X_test)
predictions_v2

In [ ]:
df_test_full["target"] = predictions_v2[:,1]

prepared_df_catboost_v2 = df_test_full[["variantid1", "variantid2", "target"]]
prepared_df_catboost_v2.head(2)

In [ ]:
prepared_df_catboost_v2.to_csv("catboost_prepared_dataset_v2.csv", 
                               index=False)